## Version information

This notebook was created using Python 3.10 and the following package versions:

- pandas 2.2.1
- numpy 1.26.1
- 

## Moving Data

Before moving data, we need to get data from US CDC:

`wget --mirror --convert-links --adjust-extension --page-requisites --no-parent https://wonder.cdc.gov/nndss/static/`

In [9]:
# save html file from url
import requests
from multiprocessing import Pool
import multiprocessing

def get_website(week):
  year = 2015
  url = f"https://wonder.cdc.gov/nndss/nndss_weekly_tables_{year}.asp?mmwr_year={year}&mmwr_week={week}&mmwr_table=2I&request=Submit"

  output_file = f'./US/wonder.cdc.gov/nndss/static/{year}/month/{year}_{week}.html'  # Name of the file to save the HTML content

  response = requests.get(url)

  if response.status_code == 200:
      with open(output_file, 'w', encoding='utf-8') as f:
          f.write(response.text)
  else:
      print('Failed to retrieve HTML content. Status code:', response.status_code)


In [10]:
get_website('01')

In [11]:
year = 2015
weeks = list(range(1, 53))
# trans weeks from 1 to 01
weeks = [f'{week:02}' for week in weeks]

num_processes = int(0.1 * multiprocessing.cpu_count())
with Pool(processes=num_processes) as pool:
  results_1 = pool.map(get_website, weeks)

In [19]:
import os
import shutil

# Function to recursively search for HTML files in a directory
def find_html_files(directory):
    html_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.html'):
                html_files.append(os.path.join(root, file))
    
    # Remove path contains exclude_dir str
    
    return html_files

# Source directory where HTML files are located
source_directory = "./US/wonder.cdc.gov/nndss/static"
destination_directory = './US/AllData'
html_files_list = find_html_files(source_directory)

# remove end with 'index.html' or web.config.html
html_files_list = [x for x in html_files_list if not x.endswith('index.html') and not x.endswith('web.config.html')]

# remove path contains annual
html_files_list = [x for x in html_files_list if 'annual' not in x]

# remove path contains figure or pdf
html_files_list = [x for x in html_files_list if 'figure' not in x and 'pdf' not in x]

## Reading Data

In [20]:
import pandas as pd
import numpy as np
import os
from io import StringIO
from html.parser import HTMLParser
import re
from datetime import datetime
from multiprocessing import Pool
import multiprocessing

In [21]:
# from text get date information
def extract_date_from_title(title):
    # Regular expression pattern for matching dates in the specific format
    date_pattern = re.compile(r'(\bJanuary|\bFebruary|\bMarch|\bApril|\bMay|\bJune|\bJuly|\bAugust|\bSeptember|\bOctober|\bNovember|\bDecember)\s+\d{1,2},\s+\d{4}')
    # Search for the pattern in the title
    match = date_pattern.search(title)
    if match:
        date_str = match.group(0)
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(date_str, '%B %d, %Y')
        return date_obj
    else:
        return None

# HTMLParser subclass for parsing titles
class TitleParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.title = None
        self.recording = False

    def handle_starttag(self, tag, attrs):
        if tag == 'title':
            self.recording = True

    def handle_data(self, data):
        if self.recording:
            self.title = data

    def handle_endtag(self, tag):
        if tag == 'title':
            self.recording = False

# Get year and week from file path
def get_year_week(file_path):
    # Regular expression pattern for matching year and week in the specific format
    year_week_pattern = re.compile(r'(\d{4})-(\d{2})')
    # Search for the pattern in the file path
    match = year_week_pattern.search(file_path)
    if match:
        year = match.group(1)
        week = match.group(2)
        return year, week
    else:
        return None, None
            
# Function to read HTML files and extract data
def read_html_files(file_name):
    # Read HTML file content
    with open(file_name, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Extract tables using pandas
    html_io = StringIO(html_content)
    try:
        df_list = pd.read_html(html_io)
        # find the table with the most rows
        df = max(df_list, key=lambda x: x.shape[1])
    except ValueError as e:
        print(f"Error reading HTML tables from {file_name}: {e}")
        df = None

    # Create parser and extract title
    parser = TitleParser()
    parser.feed(html_content)
    title = parser.title
    date = extract_date_from_title(title)
    year, week = get_year_week(file_name)

    return df, date, year, week

# Function to clean data
def clean_data(file_name):
    df, date, year, week = read_html_files(file_name)
    if df is not None:
        # get column names
        col_names = df.columns.values
        df_names = pd.DataFrame(col_names.tolist())
        df_names = df_names.drop(0)
        df_names = df_names.reset_index()
        df_names['index'] = df_names['index']

        # replace column names with column number
        df.columns = range(df.shape[1])
        df = df.melt(id_vars=[0], value_vars=range(1, df.shape[1]), var_name='State', value_name='Cases')
        df = df.rename(columns={0: 'Area'})

        # Replace - with 0 in Cases column
        # df['Cases'] = df['Cases'].replace('-', '0')
        # df['Cases'] = df['Cases'].replace('—', '0')
        # df['Cases'] = df['Cases'].replace('', '0')
        # df['Cases'] = df['Cases'].astype(int)

        # add date year and week column
        df['Date'] = date
        df['Year'] = year
        df['Week'] = week
        # convert file_name to url
        df['URL'] = file_name.replace('./US/', 'https://')

        # merge with df_names by index and State
        df = pd.merge(df, df_names, left_on='State', right_on='index', how='left')
        df = df.rename(columns={0: 'Disease'})
        # add column 2 if not present
        if 2 not in df.columns:
            df[2] = ''
        if 3 not in df.columns:
            df[3] = ''
        if 1 in df.columns:
            df = df[['Area', 'Date', 'Year', 'Week', 'Disease', 1, 2, 3, 'Cases', 'URL']]
            return df
        else:
            pass
    data = {'Area': [''], 'Date': [date], 'Year': [year], 'Week': [week], 'Disease': [''], 1: [''], 2: [''], 3: [''], 'Cases': ['No Data'], 'URL': [file_name.replace('./US/', 'https://')]}
    df = pd.DataFrame(data)
    return df

In [22]:
num_processes = int(0.9 * multiprocessing.cpu_count())
with Pool(processes=num_processes) as pool:
    results = pool.map(clean_data, html_files_list)
final_df = pd.concat(results, ignore_index=True)

/tmp/ipykernel_3645502/1874626083.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(results, ignore_index=True)


## Reading 2015 data

In [77]:
# from text get date information
def extract_date_from_title(title):
    # Regular expression pattern for matching dates in the specific format
    date_pattern = re.compile(r'(\bJanuary|\bFebruary|\bMarch|\bApril|\bMay|\bJune|\bJuly|\bAugust|\bSeptember|\bOctober|\bNovember|\bDecember)\s+\d{1,2},\s+\d{4}')
    # Search for the pattern in the title
    match = date_pattern.search(title)
    if match:
        date_str = match.group(0)
        # Convert the date string to a datetime object
        date_obj = datetime.strptime(date_str, '%B %d, %Y')
        return date_obj
    else:
        return None

# HTMLParser subclass for parsing titles
class TitleParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.title = None
        self.recording = False

    def handle_starttag(self, tag, attrs):
        if tag == 'title':
            self.recording = True

    def handle_data(self, data):
        if self.recording:
            self.title = data

    def handle_endtag(self, tag):
        if tag == 'title':
            self.recording = False

# Get year and week from file path
def get_year_week(file_path):
    # Regular expression pattern for matching year and week in the specific format
    year_week_pattern = re.compile(r'(\d{4})_(\d{2})')
    # Search for the pattern in the file path
    match = year_week_pattern.search(file_path)
    if match:
        year = match.group(1)
        week = match.group(2)
        return year, week
    else:
        return None, None
            
# Function to read HTML files and extract data
def read_html_files(file_name):
    # Read HTML file content
    with open(file_name, 'r', encoding='utf-8') as file:
        html_content = file.read()

    # Extract tables using pandas
    html_io = StringIO(html_content)
    try:
        df_list = pd.read_html(html_io)
        # find the table with the most rows
        df = max(df_list, key=lambda x: x.shape[1])
    except ValueError as e:
        print(f"Error reading HTML tables from {file_name}: {e}")
        df = None

    # Create parser and extract title
    parser = TitleParser()
    parser.feed(html_content)
    title = parser.title
    date = extract_date_from_title(title)
    year, week = get_year_week(file_name)

    return df, date, year, week

# Function to clean data
def clean_data(file_name):
    df, date, year, week = read_html_files(file_name)
    if df is not None:
        # column names as first 3 rows
        df_names = df.iloc[:3, :]
        df_names = pd.DataFrame(df_names).T
        df_names = df_names.reset_index()
        df_names['index'] = df_names['index']
        # drop first 3 rows
        df = df.iloc[3:, :]
        # replace column names with column number
        df.columns = range(df.shape[1])
        df = df.melt(id_vars=[0], value_vars=range(1, df.shape[1]), var_name='State', value_name='Cases')
        df = df.rename(columns={0: 'Area'})

        # Replace - with 0 in Cases column
        # df['Cases'] = df['Cases'].replace('-', '0')
        # df['Cases'] = df['Cases'].replace('—', '0')
        # df['Cases'] = df['Cases'].replace('', '0')
        # df['Cases'] = df['Cases'].astype(int)

        # add date year and week column
        df['Date'] = date
        df['Year'] = year
        df['Week'] = week
        # convert file_name to url
        df['URL'] = file_name.replace('./US/', 'https://')

        # merge with df_names by index and State
        df = pd.merge(df, df_names, left_on='State', right_on='index', how='left')
        df = df.rename(columns={0: 'Disease'})
        # add column 2 if not present
        if 2 not in df.columns:
            df[2] = ''
        if 3 not in df.columns:
            df[3] = ''
        if 1 in df.columns:
            df = df[['Area', 'Date', 'Year', 'Week', 'Disease', 1, 2, 3, 'Cases', 'URL']]
            return df
        else:
            pass
    data = {'Area': [''], 'Date': [date], 'Year': [year], 'Week': [week], 'Disease': [''], 1: [''], 2: [''], 3: [''], 'Cases': ['No Data'], 'URL': [file_name.replace('./US/', 'https://')]}
    df = pd.DataFrame(data)
    return df

In [78]:
source_directory = "./US/month"
html_files_list = find_html_files(source_directory)

In [79]:
num_processes = int(0.9 * multiprocessing.cpu_count())
with Pool(processes=num_processes) as pool:
    results_1 = pool.map(clean_data, html_files_list)
final_df_1 = pd.concat(results_1, ignore_index=True)

In [ ]:
# row bind final_df_1 and final_df
final_df = pd.concat([final_df_1, final_df], ignore_index=True)

# save to csv
final_df.to_csv('./US/AllData.csv', index=False)

# Filter Disease contains pertussis
pertussis_df = final_df[final_df['Disease'].str.contains('pertussis', case=False)]
pertussis_df = pertussis_df[pertussis_df[1].str.contains('Current', case=False)]
# filter Area is in Total, United States, UNITED STATES
pertussis_df = pertussis_df[pertussis_df['Area'].isin(['Total', 'United States', 'UNITED STATES'])]

# Arrange by Year and Week
pertussis_df = pertussis_df.sort_values(by=['Date'])
# reindex
pertussis_df = pertussis_df.reset_index(drop=True)
# save to csv
pertussis_df.to_csv('./US/pertussis.csv', index=False)
# pertussis_df